<a href="https://colab.research.google.com/github/kplam624/Mask-Finder/blob/imagepreprocessing/image_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import dependencies
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np

import PIL

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import (
    VGG19,
    preprocess_input,
    decode_predictions
)

In [20]:
!pip install split-folders 
import splitfolders

In [2]:
names = ['with_mask', 'without_mask']
batch = 32

In [3]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)
train_datagen

In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
Data = '/content/drive/MyDrive/Colab Notebooks/maskfinder/Data'

In [22]:
# Line below will split the folders into test, validation, and test data.
# Only run when first using the notebook!!

# splitfolders.ratio(Data, output="output", seed=1337, ratio=(.8, 0.1,0.1))

Copying files: 11043 files [26:32,  6.93 files/s]


In [24]:
train_data = '/content/output/train'
test_data = '/content/output/test'
val_data = '/content/output/val'

In [25]:
train_generator = train_datagen.flow_from_directory(
    directory = train_data,
    target_size = (220,220),
    batch_size = batch,
    class_mode = 'categorical')

Found 8833 images belonging to 2 classes.


In [27]:
val_generator = train_datagen.flow_from_directory(
    directory = val_data,
    target_size = (220,220),
    batch_size = batch,
    class_mode = 'categorical')

Found 1104 images belonging to 2 classes.


In [11]:
from keras import Sequential
from keras.layers import Flatten, Dense

In [12]:
vgg19 = VGG19(weights = "imagenet", include_top = False, input_shape = (220,220,3))

for layers in vgg19.layers:
    layers.trainable = False

80142336/80134624 [==============================] - 1s 0us/step


In [13]:
model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dense(units = 2, activation = 'sigmoid'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 6, 6, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 36866     
Total params: 20,061,250
Trainable params: 36,866
Non-trainable params: 20,024,384
_________________________________________________________________


In [15]:
model.compile(optimizer = 'Adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [17]:
try:
    model.fit(train_generator,
              steps_per_epoch = len(train_generator)//32,
              epochs = 10,
              validation_data = test_generator,
              validation_steps=len(test_generator)//32,
              verbose = 1
             )
except 'UnidentifiedImageError':
    print('oops')

Epoch 1/10
8/8 [==============================] - 213s 27s/step - loss: 0.8881 - accuracy: 0.6562 - val_loss: 0.3159 - val_accuracy: 0.8594
Epoch 2/10
8/8 [==============================] - 206s 26s/step - loss: 0.4243 - accuracy: 0.8594 - val_loss: 0.1097 - val_accuracy: 0.9844
Epoch 3/10
8/8 [==============================] - 205s 26s/step - loss: 0.2466 - accuracy: 0.9141 - val_loss: 0.0709 - val_accuracy: 0.9844
Epoch 4/10
8/8 [==============================] - 203s 25s/step - loss: 0.2242 - accuracy: 0.9219 - val_loss: 0.0870 - val_accuracy: 0.9688
Epoch 5/10
8/8 [==============================] - 204s 26s/step - loss: 0.1807 - accuracy: 0.9414 - val_loss: 0.0385 - val_accuracy: 1.0000
Epoch 6/10
8/8 [==============================] - 202s 25s/step - loss: 0.1543 - accuracy: 0.9492 - val_loss: 0.1132 - val_accuracy: 0.9531
Epoch 7/10
8/8 [==============================] - 202s 25s/step - loss: 0.1408 - accuracy: 0.9375 - val_loss: 0.0372 - val_accuracy: 0.9844
Epoch 8/10
8/8 [====